In [7]:
def generate_biagrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
        
    return x

In [8]:
#examples

generate_biagrams(['This' , 'film' , 'is' , 'terrible'])

['This', 'film', 'is', 'terrible', 'is terrible', 'This film', 'film is']

In [9]:
import torch
from torchtext import data
from torchtext import datasets

In [40]:
SEED=42


#if we are using preprocessing we dont need include lengths
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Text = data.Field(tokenize='spacy' , preprocessing=generate_biagrams)
Label = data.LabelField(dtype=torch.float)

F:\anaconda\lib\site-packages\torchtext\data\field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
F:\anaconda\lib\site-packages\torchtext\data\field.py:150: UserWarning: LabelField class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [42]:
import random

train_data, test_data = datasets.IMDB.splits(Text , Label)

train_data , valid_data = train_data.split(random_state=random.seed(SEED))

F:\anaconda\lib\site-packages\torchtext\data\example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)


In [43]:
#build max vocab size and trainset

Max_vocab_size = 25000

Text.build_vocab(train_data , 
                max_size = Max_vocab_size , 
                vectors = 'glove.6B.100d' , 
                unk_init = torch.Tensor.normal_)


Label.build_vocab(train_data)

In [86]:
print(Label.vocab.stoi)

defaultdict(None, {'pos': 0, 'neg': 1})


In [44]:
Batch_size = 64


train_iterator , valid_iterator , test_iterator = data.BucketIterator.splits(
(train_data , valid_data  , test_data) , 
batch_size=Batch_size)

F:\anaconda\lib\site-packages\torchtext\data\iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


# Build the model

In [45]:
import torch.nn as nn

import torch.nn.functional as F


In [46]:
class FastText(nn.Module):
    def __init__(self , vocab_size , embedding_dim , output_dim , pad_idx):
        
        super().__init__()
        
        self.embedding  = nn.Embedding(vocab_size , embedding_dim , padding_idx=pad_idx)
        
        self.fc = nn.Linear(embedding_dim  , output_dim)
        
    def forward(self, text):
        
        #text  = [sent len , batch size]
        
        embedded  = self.embedding(text)
        
        #embedded= [sent len , batch size , emb dim]
        
        embedded = embedded.permute(1,0,2)
        
        #embedded = [batch size ,  sent len , emb dim]
        
        pooled = F.avg_pool2d(embedded , (embedded.shape[1] , 1)).squeeze(1)
        
        
        #pooled = [batch size , emb dim]
        
        return self.fc(pooled)
        

In [50]:
INPUT_DIM = len(Text.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1
PAD_IDX = Text.vocab.stoi[Text.pad_token]

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

In [63]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,500,301 trainable parameters


In [65]:
pretrained_embeddings = Text.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 1.9269,  1.4873,  0.9007,  ...,  0.1233,  0.3499,  0.6173],
        [ 0.7262,  0.0912, -0.3891,  ...,  0.0821,  0.4440, -0.7240],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-1.4633,  1.0004,  0.4673,  ...,  0.7877,  0.5888, -1.7355],
        [ 0.1919,  0.4564,  0.2600,  ..., -0.8075,  1.4772, -1.4115],
        [-0.8232,  0.8251,  0.2498,  ...,  0.6848,  1.1238,  0.3143]])

In [66]:
UNK_IDX = Text.vocab.stoi[Text.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

# train the model

In [67]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [68]:
criterion = nn.BCEWithLogitsLoss()


In [69]:


def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc



In [70]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [71]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [72]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [74]:


N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

F:\anaconda\lib\site-packages\torchtext\data\batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 01 | Epoch Time: 1m 24s
	Train Loss: 0.687 | Train Acc: 62.63%
	 Val. Loss: 0.634 |  Val. Acc: 71.31%
Epoch: 02 | Epoch Time: 1m 22s
	Train Loss: 0.649 | Train Acc: 74.81%
	 Val. Loss: 0.499 |  Val. Acc: 76.85%
Epoch: 03 | Epoch Time: 1m 13s
	Train Loss: 0.574 | Train Acc: 80.12%
	 Val. Loss: 0.430 |  Val. Acc: 80.25%
Epoch: 04 | Epoch Time: 1m 13s
	Train Loss: 0.497 | Train Acc: 84.27%
	 Val. Loss: 0.396 |  Val. Acc: 83.47%
Epoch: 05 | Epoch Time: 1m 12s
	Train Loss: 0.431 | Train Acc: 87.13%
	 Val. Loss: 0.389 |  Val. Acc: 85.29%


In [75]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.390 | Test Acc: 85.33%


In [83]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = generate_biagrams([tok.text for tok in nlp.tokenizer(sentence)])
    indexed = [Text.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [84]:
predict_sentiment(model, "This film is terrible")

1.0

In [85]:
predict_sentiment(model, "This film is great")

8.304921117936262e-22